# Question: Does the company have ethical corporate governance principles?

The quick definition of "corporate governance" is "the rules, practices, and processes by which a company is directed and controlled." In less dry terms, issues we look at in the corporate-governance area include the quality and composition of companies' management teams and boards of directors, and how they interact with stakeholders, including investors.

One major area to dig into is executive and CEO pay. Are the CEO and other top executives compensated fairly and according to performance measures? Pay plans that reflect actual performance are a positive mark, and egregiously overpaid CEOs are a negative. ESG investors should also frown on outrageous perks enjoyed by executives on the shareholder dime, like private jets and expensive club memberships.

ESG investors should also examine each company's board of directors. Is the board diverse, composed of folks who are able to push back against management when necessary? Examine aspects like director tenure -- long tenure can lead to a less diverse, robust board. The purpose of the directors is to be a voice for shareholders, so it's not good to see an entrenched or complicit board that will rubber-stamp every whim of management.

It's a good sign when a board of directors has an independent chair. If the CEO also holds the chairperson role, it's much harder for a board to do its job properly. Combining the chairperson and CEO roles can make the leader more like a monarch than a company employee, which is what a CEO actually is.

Another point to consider: Does the company have multiple classes of stock, with one or more classes controlled by executives who have supervoting rights? Supervoting rights mean those shares held by management or insiders can have many more votes than those held by regular shareholders; while a normal shareholder has one vote per share, management-controlled supervoting shares might get 10 votes per share. That would indicate that ordinary shareholders have little voice in votes, a shareholder-unfriendly scenario.

Companies with a collection of poor corporate-governance policies are red-flagged and possibly disqualified from inclusion in our ESG portfolio.

# Data Sources

**Women % Board Representation**

https://2020wob.com/educate2/

**CEO Pay Ratio**

https://www.salary.com/personal/executive-salaries/

# Imports

In [37]:
# All necessary imports
from bs4 import BeautifulSoup
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import pandas as pd
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 100)

import re
import time
import pickle

import warnings
warnings.simplefilter("ignore")

# Getting Companies

In [38]:
# Load in pickled list of companies and corresponding tickers
with open('./generated_data/companies_n_tickers.pickle','rb') as f:
    companies_n_tickers = pickle.load(f)

# Getting Data on Women's Board Representation

In [55]:
'''
Because of the likelihood and trouble getting blocked by the site, 
I used randomized delays to decrease the bot-like nature of the scraping tool.
'''
delays = [5, 2, 4, 7, 8, 9]
delay = np.random.choice(delays)

In [56]:
def get_females_on_board(companies):
    
    # Create empty data frame for scraping data
    perc_female_board = pd.DataFrame(columns=['company','percent_female_on_board'])

    # Set path to chromedriver
    PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
    # Define options 
    options = Options()
    # Remove pop up window
    options.add_argument("--headless")
    # Define driver
    driver = webdriver.Chrome(PATH, options=options)
    # # Define driver
    # driver = webdriver.Chrome(PATH)
    driver.set_window_size(1080,800)
    # Define url
    url= "https://2020wob.com/educate2/"
    # Get website
    driver.get(url)
    # Sleep 5 seconds
    time.sleep(5)

    # Loop through companies    
    for a,b in companies:
        try:
            # Find search bar
            search_bar = driver.find_element_by_xpath('//*[@id="CName"]')
            # Clear search bar
            search_bar.clear()
            # Enter company name into search bar
            search_bar.send_keys(a)
            # Time delay
            time.sleep(delay)
            try:
                # Click company in dropdown
                driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
            except:
                # Clear search bar
                search_bar.clear()
                # Enter company name into search bar
                search_bar.send_keys(a)
                # Define dropdown options of search
                dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
                # Check if no options
                if dropdown_options == '':
                    # Split company into list
                    split_company = a.split()
                    # Clear search bar
                    search_bar.clear()
                    # Enter company name into search bar
                    search_bar.send_keys(split_company[0])
                    # Time delay
                    time.sleep(delay)
                    # Define dropdown options of search
                    dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
                    # Check if single option in dropdown
                    if dropdown_options.count('\n') == 0:
                        # Click single dropdown option
                        driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
                    # Check if more than one option in dropdown
                    elif dropdown_options.count('\n') > 0:
                        # Clear search bar
                        search_bar.clear()
                        # Enter company name into search bar
                        search_bar.send_keys(split_company[0],' ',split_company[1])
                        # Time delay
                        time.sleep(delay)
                        # Define dropdown options of search
                        dropdown_options = driver.find_element_by_xpath('//*[@id="ui-id-1"]').text
                        # Check if single option in dropdown
                        if dropdown_options.count('\n') == 0:
                            # Click single dropdown option
                            driver.find_element_by_xpath('//*[@id="ui-id-1"]').click()
                        else:
                            pass
            # Time delay
            time.sleep(delay)
            # Enter
            search_bar.send_keys(Keys.ENTER)
            # Time delay
            time.sleep(delay)
            # Get percent women
            percent_women = int((driver.find_element_by_xpath('//*[@id="genderdiversityindex"]/div/div/table/tbody/tr/td[5]').text).replace('%','')) / 100
            # Time delay
            time.sleep(delay)

            # Append values to data frame 
            perc_female_board = perc_female_board.append({'company':a,
                                                          'percent_female_on_board':percent_women}, ignore_index=True)

        except:
            # Append null value to data frame
            perc_female_board = perc_female_board.append({'company':a,
                                                          'percent_female_on_board':0}, ignore_index=True)
    # Sort data frame by value descending
    perc_female_board = perc_female_board.sort_values(by='percent_female_on_board', ascending=False)

    return perc_female_board


In [57]:
# Define fem_board_df using function
fem_board_df = get_females_on_board(companies_n_tickers)

In [58]:
# Check fem_board_df
fem_board_df

,company,percent_female_on_board
344,Omnicom Group,0.67
64,Best Buy Co. Inc.,0.60
444,Ulta Beauty,0.54
205,General Motors,0.54
461,ViacomCBS,0.54
...,...,...
42,APA Corporation,0.00
41,A.O. Smith Corp,0.00
460,VF Corporation,0.00
462,Viatris,0.00


# CEO Pay Ratio

In [59]:
def get_ceo_pay_ratio(companies):
    
    ceo_pay_ratios = pd.DataFrame(columns=['company','ceo_pay_ratio'])
    
    # Set path to chromedriver
    PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
    # Define options 
    options = Options()
    # Remove pop up window
    options.add_argument("--headless")
    # Define driver
    driver = webdriver.Chrome(PATH, options=options)
    # Window size
    driver.set_window_size(1080,800)
    # Define url
    url= "https://www1.salary.com/APPLE-INC-Executive-Salaries.html"
    # Get website
    driver.get(url)
    # Time delay
    time.sleep(5)
    
    for a,b in companies:
        try:
            # Find search bar
            search_bar = driver.find_element_by_xpath('//*[@id="searchform"]/div/input')
            # Time delay
            time.sleep(delay)
            # Clear search bar
            search_bar.clear()
            # Enter company name into search bar
            search_bar.send_keys(a)
            # Time delay
            time.sleep(delay)
            # Search company
            search_bar.send_keys(Keys.ENTER)
            # Time delay
            time.sleep(delay)
            # Click on company
            driver.find_element_by_xpath('//*[@id="top"]/div[3]/div/div[1]/div[4]/table/tbody/tr[1]/td/span/a').click()
            # Time delay
            time.sleep(delay)
            # Get ceo pay ratio
            pay_ratio = int((driver.find_element_by_xpath('//*[@id="top"]/div[3]/div/div[1]/div[17]/table/tbody/tr/td[4]/span').text).split(':')[0])
            # Time delay
            time.sleep(delay)

            # Append value to data frame
            ceo_pay_ratios = ceo_pay_ratios.append({'company': a,
                                                    'ceo_pay_ratio': pay_ratio}, ignore_index=True)
            
        except:
            # Define url
            url= "https://www1.salary.com/APPLE-INC-Executive-Salaries.html"
            # Get website
            driver.get(url)
            # Time delay
            time.sleep(delay)
            # Append value to data frame
            ceo_pay_ratios = ceo_pay_ratios.append({'company': a,
                                                    'ceo_pay_ratio': 10000}, ignore_index=True)

    ceo_pay_ratios = ceo_pay_ratios.sort_values(by='ceo_pay_ratio')
    
    return ceo_pay_ratios
    

In [60]:
# Define ceo_pay_ratios using function
ceo_pay_ratios = get_ceo_pay_ratio(companies_n_tickers)

In [61]:
# Check ceo_pay_ratios
ceo_pay_ratios

,company,ceo_pay_ratio
440,"Twitter, Inc.",0
429,"Tesla, Inc.",0
170,Etsy,6
63,Berkshire Hathaway,6
47,Arista Networks,9
...,...,...
346,Oracle Corp.,10000
347,Otis Worldwide,10000
348,Paccar,10000
335,Norwegian Cruise Line Holdings,10000
